### Intro

In [ ]:
import csv
import os
from datetime import datetime

USERS_FILE = "source_files/users.csv"
MOVIES_FILE = "source_files/movies.csv"
BORROW_FILE = "source_files/borrowings.csv"


# ================================
# CSV HELPERS
# ================================

def load_csv(filename):
    if not os.path.exists(filename):
        return []

    with open(filename, newline="", encoding="utf-8") as file:
        reader = csv.DictReader(file)
        return list(reader)


def save_csv(filename, data, fieldnames):
    with open(filename, "w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)


def generate_new_id(records):
    if not records:
        return 1
    return max(int(r["id"]) for r in records) + 1




# ================================
# USER MANAGEMENT
# ================================

def add_user():
    users = load_csv(USERS_FILE)
    name = input("Enter user name: ").strip()
    if not name:
        print("Name cannot be empty.")
        return

    user_id = generate_new_id(users)
    users.append({"id": user_id, "user name": name})
    save_csv(USERS_FILE, users, ["id", "user name"])

    print(f"User added with ID: {user_id}")


def view_users():
    users = load_csv(USERS_FILE)
    uesrs = []
    if not users:
        print("No users found.")
        return

    print("\n--- USERS ---")
    for u in users:
        print(f"{u['id']}: {u['name']}")
    print("-------------")




# ================================
# MOVIE MANAGEMENT
# ================================

def add_movie():
    movies = load_csv(MOVIES_FILE)

    title = input("Enter movie title: ").strip()
    director = input("Enter director: ").strip()
    year = input("Enter release year: ").strip()
    copies = input("Enter number of copies: ").strip()

    if not title or not director or not year or not copies.isdigit():
        print("Invalid input.")
        return

    movie_id = generate_new_id(movies)

    movies.append({
        "id": movie_id,
        "title": title,
        "director": director,
        "year": year,
        "copies": copies
    })

    save_csv(MOVIES_FILE, movies,
             ["id", "title", "director", "year", "copies"])

    print(f"Movie added with ID: {movie_id}")


def view_movies():
    movies = load_csv(MOVIES_FILE)
    if not movies:
        print("No movies found.")
        return

    print("\n--- MOVIES ---")
    for m in movies:
        print(f"{m['id']} | {m['title']} | {m['director']} | {m['year']} | Copies: {m['copies']}")
    print("--------------")



#borrow function
def borrow_movies():
    users = load_csv(USERS_FILE)
    movies = load_csv(MOVIES_FILE)
    borrowings = load_csv(BORROW_FILE)

    user_id = input("User ID: ").strip()
    if not any(str(u["id"]) == user_id for u in users):
        print("User not found.")
        return

    movie_ids = [mid.strip() for mid in input("Enter Movie IDs (comma separated): ").split(",") if mid.strip()]
    movies_dict = {str(m["id"]): m for m in movies}
    
    # Check all requested movies for validity and availability
    unavailable = []
    for mid in movie_ids:
        movie = movies_dict.get(mid)
        if not movie:
            unavailable.append(f"{mid} (not found)")
        elif int(movie["copies"]) <= 0:
            unavailable.append(f"{mid} ({movie['title']} - no copies)")
    
    if unavailable:
        print("\nUnavailable:")
        for u in unavailable:
            print(" -", u)
        print("No movies have been borrowed.")
        return    # abort if ANY are unavailable
    
    # ALL requested movies are available -> proceed to borrow
    for mid in movie_ids:
        movies_dict[mid]["copies"] = str(int(movies_dict[mid]["copies"]) - 1)

    # Update/add borrowing record for the user
    record = next((b for b in borrowings if b["user_id"] == user_id), None)
    if record:
        current = set(record["movie_ids"].split(",")) if record["movie_ids"].strip() else set()
        new_borrowed = current.union(movie_ids)
        record["movie_ids"] = ",".join(new_borrowed)
        # Optionally, update date to current
        record["date"] = datetime.now().strftime("%Y-%m-%d")
    else:
        borrowings.append({
            "user_id": user_id,
            "movie_ids": ",".join(movie_ids),
            "date": datetime.now().strftime("%Y-%m-%d")
        })
    
    # Save updates
    save_csv(MOVIES_FILE, list(movies_dict.values()), ["id", "title", "director", "year", "copies"])
    save_csv(BORROW_FILE, borrowings, ["user_id", "movie_ids", "date"])
    
    print("Movies borrowed successfully!")


#return
def return_movies():
    movies = load_csv(MOVIES_FILE)
    movies_dict = {str(m["id"]): m for m in movies}
    borrowings = load_csv(BORROW_FILE)

    user_id = input("User ID: ").strip()
    record = next((b for b in borrowings if b["user_id"] == user_id), None)
    if not record:
        print("User has no borrowings.")
        return

    to_return = [mid.strip() for mid in input("Enter Movie IDs to return: ").split(",") if mid.strip()]
    borrowed = set(record["movie_ids"].split(",")) if record["movie_ids"].strip() else set()
    actually_returned = []

    for mid in to_return:
        if mid in borrowed:
            # Only update if the movie exists (robustness)
            if mid in movies_dict:
                movies_dict[mid]["copies"] = str(int(movies_dict[mid]["copies"]) + 1)
            borrowed.remove(mid)
            actually_returned.append(mid)

    if not actually_returned:
        print("No valid movies returned.")
        return

    if borrowed:
        record["movie_ids"] = ",".join(borrowed)
    else:
        # All movies returned, so remove the borrowing record
        borrowings = [b for b in borrowings if b["user_id"] != user_id]

    save_csv(MOVIES_FILE, list(movies_dict.values()),
             ["id", "title", "director", "year", "copies"])
    # Remove any records where movie_ids is empty
    borrowings = [b for b in borrowings if b.get("movie_ids", "").strip()]
    save_csv(BORROW_FILE, borrowings, ["user_id", "movie_ids", "date"])

    print("Movies returned successfully!")

    #list borrowed movies for a user

def list_borrowed_movies():
    movies = load_csv(MOVIES_FILE)
    movies_dict = {str(m['id']): m for m in movies}
    borrowings = load_csv(BORROW_FILE)

    user_id = input("User ID: ").strip()
    record = next((b for b in borrowings if b["user_id"] == user_id), None)
    if not record or not record.get("movie_ids", "").strip():
        print("No borrowings found for this user.")
        return

    borrowed_ids = [mid for mid in record["movie_ids"].split(",") if mid.strip()]
    print("\n--- Borrowed Movies ---")
    print("Borrowing Date:", record.get("date", "N/A"))
    for mid in borrowed_ids:
        movie = movies_dict.get(mid)
        if movie:
            print(f"{mid}: {movie['title']}")
        else:
            print(f"{mid}: [Movie not found]")
    print("-----------------------")




# ================================
# MENU
# ================================
def main_menu():
    while True:
        print("\n===== MOVIE MANAGEMENT SYSTEM (CSV) =====")
        print("1. Add User")
        print("2. View Users")
        print("3. Add Movie")
        print("4. View Movies")
        print("5. Borrow Movies")
        print("6. Return Movies")
        print("7. List Borrowed Movies")
        print("8. Exit")

        choice = input("Choose option: ")

        if choice == "1":
            add_user()
        elif choice == "2":
            view_users()
        elif choice == "3":
            add_movie()
        elif choice == "4":
            view_movies()
        elif choice == "5":
            borrow_movies()
        elif choice == "6":
            return_movies()
        elif choice == "7":
            list_borrowed_movies()
        elif choice == "8":
            break
        else:
            print("Invalid option.")


main_menu()





===== MOVIE MANAGEMENT SYSTEM (CSV) =====
1. Add User
2. View Users
3. Add Movie
4. View Movies
5. Borrow Movies
6. Return Movies
7. List Borrowed Movies
8. Exit
[{'User ID': '1', 'User Name': 'Alice'}, {'User ID': '2', 'User Name': 'Bob'}, {'User ID': '3', 'User Name': 'Charlie'}, {'User ID': '4', 'User Name': 'David'}, {'User ID': '5', 'User Name': 'Eve'}, {'User ID': '6', 'User Name': 'Frank'}, {'User ID': '7', 'User Name': 'Grace'}, {'User ID': '8', 'User Name': 'Heidi'}, {'User ID': '9', 'User Name': 'Ivan'}, {'User ID': '10', 'User Name': 'Judy'}]

--- USERS ---


KeyError: 'id'